In [1]:
from sklearn.datasets import load_iris
import numpy as np
import math
from tqdm import tqdm
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    OrdinalEncoder,
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix

In [2]:
model = LogisticRegression(C=0.5)

In [3]:
iden = pd.read_csv('train_identity.csv')

In [4]:
trans = pd.read_csv('train_transaction.csv')

In [5]:
joined = trans.merge(iden,how='left')

In [6]:
del iden

In [7]:
del trans

In [8]:
joined = joined[joined['TransactionDT'].notnull()]

In [9]:
joined = joined[joined['isFraud'].notnull()]

In [10]:
len(joined)

590540

In [11]:
limite = np.percentile(joined['TransactionDT'], 80)

In [12]:
limite

12192853.600000001

In [13]:
joined['TransactionDT'].value_counts()

9474817     8
4397066     5
11576951    5
7236588     5
8218707     4
           ..
11481962    1
14956586    1
12853289    1
7622695     1
9943554     1
Name: TransactionDT, Length: 573349, dtype: int64

In [14]:
filtro1 = joined['TransactionDT'] > limite

In [20]:
filtro1.value_counts()

False    472432
True     118108
Name: TransactionDT, dtype: int64

In [15]:
validation = joined[filtro1]

In [18]:
validation

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
472432,3459432,1,12192900,33.261,C,9300,103.0,185.0,visa,138.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,NaN
472433,3459433,0,12192911,52.811,C,8809,179.0,106.0,visa,137.0,...,chrome generic for android,NaN,NaN,NaN,F,F,T,F,mobile,F3311
472434,3459434,0,12192913,136.956,C,10819,555.0,185.0,visa,226.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
472435,3459435,0,12193040,136.956,C,9633,130.0,185.0,visa,138.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
472436,3459436,0,12193199,25.000,H,17188,321.0,150.0,visa,226.0,...,firefox 59.0,24.0,3200x1800,match_status:2,T,F,T,T,desktop,Windows
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.000,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.500,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.950,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.000,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
filtro2 = joined['TransactionDT']<= limite

In [17]:
train = joined[filtro2]

In [21]:
train

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472427,3459427,0,12192667,43.95,W,15484,418.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472428,3459428,0,12192736,49.00,W,17188,321.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472429,3459429,0,12192742,40.00,H,16659,170.0,150.0,visa,226.0,...,chrome 65.0,24.0,1440x900,match_status:2,T,F,T,T,desktop,MacOS
472430,3459430,0,12192743,15.00,W,7919,194.0,150.0,mastercard,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
label = train[['isFraud']]

In [23]:
del(train['isFraud'])

In [24]:
del(train['TransactionID'])

In [25]:
del joined

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472432 entries, 0 to 472431
Columns: 432 entries, TransactionDT to DeviceInfo
dtypes: float64(399), int64(2), object(31)
memory usage: 1.5+ GB


In [27]:
ohe = {}

In [28]:
ohe['id_34'] = OneHotEncoder(handle_unknown='ignore')
id_34_encoded = (ohe['id_34'].fit_transform(train[['id_34']].astype(str)).todense().astype(int))
id_34_encoded = pd.DataFrame(id_34_encoded).add_prefix('id_34_')
train = pd.concat([train, id_34_encoded], axis=1)

In [29]:
train['id_34_0'].fillna(0,inplace=True)

In [30]:
train['id_34_1'].fillna(0,inplace=True)

In [31]:
train['id_34_2'].fillna(0,inplace=True)

In [32]:
train['id_34_3'].fillna(0,inplace=True)

In [33]:
train['id_34_4'].fillna(0,inplace=True)

In [34]:
ohe['id_35'] = OneHotEncoder(handle_unknown='ignore')
id_35_encoded = (ohe['id_35'].fit_transform(train[['id_35']].astype(str)).todense().astype(int))
id_35_encoded = pd.DataFrame(id_35_encoded).add_prefix('id_35_')
train = pd.concat([train, id_35_encoded], axis=1)

In [35]:
train['id_35_0'].fillna(0,inplace=True)

In [36]:
train['id_35_1'].fillna(0,inplace=True)

In [37]:
train['id_35_2'].fillna(0,inplace=True)

In [38]:
ohe['id_38'] = OneHotEncoder(handle_unknown='ignore')
id_38_encoded = (ohe['id_38'].fit_transform(train[['id_38']].astype(str)).todense().astype(int))
id_38_encoded = pd.DataFrame(id_38_encoded).add_prefix('id_38_')
train = pd.concat([train, id_38_encoded], axis=1)

In [39]:
train['id_38_0'].fillna(0,inplace=True)

In [40]:
train['id_38_1'].fillna(0,inplace=True)

In [41]:
train['id_38_2'].fillna(0,inplace=True)

In [42]:
ohe['ProductCD'] = OneHotEncoder(handle_unknown='ignore')
ProductCD_encoded = (ohe['ProductCD'].fit_transform(train[['ProductCD']].astype(str)).todense().astype(int))
ProductCD_encoded = pd.DataFrame(ProductCD_encoded).add_prefix('ProductCD_')
train = pd.concat([train, ProductCD_encoded], axis=1)

In [43]:
train['ProductCD_0'].fillna(0,inplace=True)

In [44]:
train['ProductCD_1'].fillna(0,inplace=True)

In [45]:
train['ProductCD_2'].fillna(0,inplace=True)

In [46]:
train['ProductCD_3'].fillna(0,inplace=True)

In [47]:
train['ProductCD_4'].fillna(0,inplace=True)

In [ ]:
#ohe['P_emaildomain'] = OneHotEncoder(handle_unknown='ignore')
#P_emaildomain_encoded = (ohe['P_emaildomain'].fit_transform(train[['P_emaildomain']].astype(str)).todense().astype(int))
#P_emaildomain_encoded = pd.DataFrame(P_emaildomain_encoded).add_prefix('P_emaildomain_')
#train = pd.concat([train, P_emaildomain_encoded], axis=1)

In [ ]:
#train['P_emaildomain'].fillna(0,inplace=True)

In [48]:
ohe['card4'] = OneHotEncoder(handle_unknown='ignore')
card4_encoded = (ohe['card4'].fit_transform(train[['card4']].astype(str)).todense().astype(int))
card4_encoded = pd.DataFrame(card4_encoded).add_prefix('card4')
train = pd.concat([train, card4_encoded], axis=1)

In [49]:
train['card40'].fillna(0,inplace=True)

In [50]:
train['card41'].fillna(0,inplace=True)

In [51]:
train['card42'].fillna(0,inplace=True)

In [52]:
train['card43'].fillna(0,inplace=True)

In [53]:
train['card44'].fillna(0,inplace=True)

In [54]:
le = {} #genero diccionario por columna para tener un labelEnc por col
for column in train.columns:
    if train[column].dtype==object:
        le[column] = OrdinalEncoder(handle_unknown='ignore')
        train[column] = le[column].fit_transform(train[column].astype(str).values.reshape(-1, 1))

In [55]:
train.fillna(-1,inplace=True)

In [56]:
model.fit(train, label)

C:\Users\julip\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\julip\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.5)

In [57]:
label.value_counts()

isFraud
0          455833
1           16599
dtype: int64

In [58]:
print(classification_report(label, model.predict(train)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98    455833
           1       0.14      0.01      0.02     16599

    accuracy                           0.96    472432
   macro avg       0.55      0.50      0.50    472432
weighted avg       0.94      0.96      0.95    472432



In [59]:
roc_auc_score(label, model.predict_proba(train)[:,1])

0.6441431332290725

### VALIDATION

In [60]:
validation

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
472432,3459432,1,12192900,33.261,C,9300,103.0,185.0,visa,138.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,NaN
472433,3459433,0,12192911,52.811,C,8809,179.0,106.0,visa,137.0,...,chrome generic for android,NaN,NaN,NaN,F,F,T,F,mobile,F3311
472434,3459434,0,12192913,136.956,C,10819,555.0,185.0,visa,226.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
472435,3459435,0,12193040,136.956,C,9633,130.0,185.0,visa,138.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
472436,3459436,0,12193199,25.000,H,17188,321.0,150.0,visa,226.0,...,firefox 59.0,24.0,3200x1800,match_status:2,T,F,T,T,desktop,Windows
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.000,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.500,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.950,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.000,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
label2=validation['isFraud']

In [62]:
validation

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
472432,3459432,1,12192900,33.261,C,9300,103.0,185.0,visa,138.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,NaN
472433,3459433,0,12192911,52.811,C,8809,179.0,106.0,visa,137.0,...,chrome generic for android,NaN,NaN,NaN,F,F,T,F,mobile,F3311
472434,3459434,0,12192913,136.956,C,10819,555.0,185.0,visa,226.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
472435,3459435,0,12193040,136.956,C,9633,130.0,185.0,visa,138.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
472436,3459436,0,12193199,25.000,H,17188,321.0,150.0,visa,226.0,...,firefox 59.0,24.0,3200x1800,match_status:2,T,F,T,T,desktop,Windows
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.000,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.500,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.950,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.000,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
del(validation['isFraud'])

In [64]:
del(validation['TransactionID'])

In [65]:
validation

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
472432,12192900,33.261,C,9300,103.0,185.0,visa,138.0,debit,NaN,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,NaN
472433,12192911,52.811,C,8809,179.0,106.0,visa,137.0,debit,NaN,...,chrome generic for android,NaN,NaN,NaN,F,F,T,F,mobile,F3311
472434,12192913,136.956,C,10819,555.0,185.0,visa,226.0,debit,NaN,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
472435,12193040,136.956,C,9633,130.0,185.0,visa,138.0,debit,NaN,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
472436,12193199,25.000,H,17188,321.0,150.0,visa,226.0,debit,310.0,...,firefox 59.0,24.0,3200x1800,match_status:2,T,F,T,T,desktop,Windows
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,15811047,49.000,W,6550,NaN,150.0,visa,226.0,debit,272.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,15811049,39.500,W,10444,225.0,150.0,mastercard,224.0,debit,204.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,15811079,30.950,W,12037,595.0,150.0,mastercard,224.0,debit,231.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,15811088,117.000,W,7826,481.0,150.0,mastercard,224.0,debit,387.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
id_34_encoded = (ohe['id_34'].transform(validation[['id_34']].astype(str)).todense().astype(int))
id_34_encoded = pd.DataFrame(id_34_encoded).add_prefix('id_34_')
validation = pd.concat([validation, id_34_encoded], axis=1)

In [67]:
validation['id_34_0'].fillna(0,inplace=True)

In [68]:
validation['id_34_1'].fillna(0,inplace=True)

In [69]:
validation['id_34_2'].fillna(0,inplace=True)

In [70]:
validation['id_34_3'].fillna(0,inplace=True)

In [71]:
validation['id_34_4'].fillna(0,inplace=True)

In [72]:
validation

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_36,id_37,id_38,DeviceType,DeviceInfo,id_34_0,id_34_1,id_34_2,id_34_3,id_34_4
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,15811047.0,49.00,W,6550.0,NaN,150.0,visa,226.0,debit,272.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
590536,15811049.0,39.50,W,10444.0,225.0,150.0,mastercard,224.0,debit,204.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
590537,15811079.0,30.95,W,12037.0,595.0,150.0,mastercard,224.0,debit,231.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
590538,15811088.0,117.00,W,7826.0,481.0,150.0,mastercard,224.0,debit,387.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


In [66]:
id_35_encoded = (ohe['id_35'].transform(validation[['id_35']].astype(str)).todense().astype(int))
id_35_encoded = pd.DataFrame(id_35_encoded).add_prefix('id_35_')
validation = pd.concat([validation, id_35_encoded], axis=1)

In [67]:
validation['id_35_0'].fillna(0,inplace=True)

In [68]:
validation['id_35_1'].fillna(0,inplace=True)

In [69]:
validation['id_35_2'].fillna(0,inplace=True)

In [70]:
id_38_encoded = (ohe['id_38'].transform(validation[['id_38']].astype(str)).todense().astype(int))
id_38_encoded = pd.DataFrame(id_38_encoded).add_prefix('id_38_')
validation = pd.concat([validation, id_38_encoded], axis=1)

In [71]:
validation['id_38_0'].fillna(0,inplace=True)

In [72]:
validation['id_38_1'].fillna(0,inplace=True)

In [73]:
validation['id_38_2'].fillna(0,inplace=True)

In [74]:
ProductCD_encoded = (ohe['ProductCD'].transform(validation[['ProductCD']].astype(str)).todense().astype(int))
ProductCD_encoded = pd.DataFrame(ProductCD_encoded).add_prefix('ProductCD_')
validation = pd.concat([validation, ProductCD_encoded], axis=1)

In [75]:
validation['ProductCD_0'].fillna(0,inplace=True)

In [76]:
validation['ProductCD_1'].fillna(0,inplace=True)

In [77]:
validation['ProductCD_2'].fillna(0,inplace=True)

In [78]:
validation['ProductCD_3'].fillna(0,inplace=True)

In [79]:
validation['ProductCD_4'].fillna(0,inplace=True)

In [79]:
#P_emaildomain_encoded = (ohe['P_emaildomain'].transform(validation[['P_emaildomain']].astype(str)).todense().astype(int))
#P_emaildomain_encoded = pd.DataFrame(P_emaildomain_encoded).add_prefix('P_emaildomain_')
#validation = pd.concat([validation, P_emaildomain_encoded], axis=1)

In [80]:
card4_encoded = (ohe['card4'].transform(validation[['card4']].astype(str)).todense().astype(int))
card4_encoded = pd.DataFrame(card4_encoded).add_prefix('card4')
validation = pd.concat([validation, card4_encoded], axis=1)

In [81]:
validation['card40'].fillna(0,inplace=True)

In [82]:
validation['card41'].fillna(0,inplace=True)

In [83]:
validation['card42'].fillna(0,inplace=True)

In [84]:
validation['card43'].fillna(0,inplace=True)

In [85]:
validation['card44'].fillna(0,inplace=True)

In [86]:
for column in validation.columns:
    if validation[column].dtype==object:
        validation[column] = le[column].transform(validation[column].astype(str).values.reshape(-1, 1))

In [87]:
validation.fillna(-1,inplace=True)

In [91]:
print(classification_report(label2, model.predict(validation)))

ValueError: Found input variables with inconsistent numbers of samples: [118108, 590540]

In [90]:
label2

472432    1
472433    0
472434    0
472435    0
472436    0
         ..
590535    0
590536    0
590537    0
590538    0
590539    0
Name: isFraud, Length: 118108, dtype: int64

In [93]:
len(model.predict(validation))

590540

In [88]:
roc_auc_score(label2, model.predict_proba(validation)[:,1])

ValueError: multi_class must be in ('ovo', 'ovr')

### TEST

In [ ]:
idenTest = pd.read_csv('test_identity.csv')

In [ ]:
transTest = pd.read_csv('test_transaction.csv')

In [ ]:
join = transTest.merge(idenTest,how='left')

In [ ]:
join.head(2)

In [ ]:
join.columns

In [ ]:
id_34_encoded = (ohe.transform(join[['id-34']].astype(str)).todense().astype(int))
id_34_encoded = pd.DataFrame(id_34_encoded).add_prefix('id_34_')
join = pd.concat([join, id_34_encoded], axis=1)

In [ ]:
join['id-34'].fillna(0,inplace=True)

In [ ]:
id_35_encoded = (ohe.transform(join[['id-35']].astype(str)).todense().astype(int))
id_35_encoded = pd.DataFrame(id_35_encoded).add_prefix('id_35_')
join = pd.concat([join, id_35_encoded], axis=1)

In [ ]:
join['id-35'].fillna(0,inplace=True)

In [ ]:
id_38_encoded = (ohe.transform(join[['id-38']].astype(str)).todense().astype(int))
id_38_encoded = pd.DataFrame(id_38_encoded).add_prefix('id_38_')
join = pd.concat([join, id_38_encoded], axis=1)

In [ ]:
join['id-38'].fillna(0,inplace=True)

In [ ]:
ProductCD_encoded = (ohe.transform(join[['ProductCD']].astype(str)).todense().astype(int))
ProductCD_encoded = pd.DataFrame(ProductCD_encoded).add_prefix('ProductCD_')
join = pd.concat([join, ProductCD_encoded], axis=1)

In [ ]:
join['ProductCD'].fillna(0,inplace=True)

In [ ]:
P_emaildomain_encoded = (ohe.transform(join[['P_emaildomain']].astype(str)).todense().astype(int))
P_emaildomain_encoded = pd.DataFrame(P_emaildomain_encoded).add_prefix('P_emaildomain_')
join = pd.concat([join, P_emaildomain_encoded], axis=1)

In [ ]:
join['P_emaildomain'].fillna(0,inplace=True)

In [ ]:
card4_encoded = (ohe.transform(join[['card4']].astype(str)).todense().astype(int))
card4_encoded = pd.DataFrame(card4_encoded).add_prefix('card4')
join = pd.concat([join, card4_encoded], axis=1)

In [ ]:
join['card4'].fillna(0,inplace=True)

In [ ]:
las = {}
for column in join.columns:
    if join[column].dtype==object:
        las[column] = LabelEncoder()
        join[column] = las[column].transform(join[column].astype(str))

In [ ]:
join.fillna(-1,inplace=True)

In [ ]:
join

In [ ]:
sub = pd.read_csv('sample_submission.csv')

In [ ]:
sub.head(2)

In [ ]:
join['predict'] = model.predict_proba(join)[:,1]

In [ ]:
set(join.columns)-set(train.columns)

In [ ]:
set(train.columns)-set(join.columns)